In [17]:
# Useful starting lines
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as m
from preprocessing import *
from plots import *
from crossval import *
from gradient_descent import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [18]:
from proj1_helpers import *
#DATA_TRAIN_PATH = 'C:/Users/joeld/Desktop/EPFL/machine learning/AIAIaie/data/train.csv' # TODO: download train data and supply path here 
DATA_TRAIN_PATH = '/Users/benoithohl/Desktop/epfl/master_epfl/Ma3/Machine_learning/AIAIaie/data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
data = pd.read_csv(DATA_TRAIN_PATH)
pd.set_option('display.max_columns', 50)
data.head(12)

In [ ]:
print("shape of data tX: ",tX.shape)
print("**")
print(len(tX[:,1]))

print(tX[0:30,0])
print("**")

# PREPROCESSING

In [19]:
Data = remove_features_with_too_many_missing_values(tX,0.66)
Data = replace_missing_values_with_global_mean(Data)
ZData = Z_score_of_each_feature(Data)




7 features removed (features number:  [ 4  5  6 12 26 27 28] )
new shape of data: (250000, 23)
range(0, 23)


In [5]:
#Visualization
Data_df = pd.DataFrame(ZData)
Data_df.head(12)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,3.149107e-01,0.068332,0.407680,-0.469966,0.882478,1.033099,0.339894,0.170929,1.277084,-0.270811,...,1.812288,-1.352820,-0.756757,-0.147267,0.386847,1.044402,-0.369921,1.557298e+00,3.248244e-01,0.412510
1,7.408270e-01,0.552505,0.540136,-0.153167,1.404888,-0.756027,-0.287584,-0.661279,1.292164,0.147536,...,0.411475,0.032730,0.090798,-1.051683,-0.357719,0.021305,-0.821154,5.267049e-01,8.329932e-01,-0.273820
2,-5.388023e-16,3.195156,1.096560,-0.349710,0.989770,-0.430168,0.340361,2.768174,1.292164,-0.292406,...,-0.737951,0.555132,0.382001,-1.200672,0.400135,0.021305,-0.863173,1.487145e+00,-1.434550e+00,-0.293970
3,4.179442e-01,0.910379,-0.005853,-0.903016,1.196690,-0.830735,-0.712705,1.084818,-0.969095,-0.716598,...,-0.397234,-1.730447,-0.323312,0.038692,-0.978149,-1.001792,0.000000,1.003673e-17,1.111175e-17,-0.745439
4,1.023804e+00,-0.914556,1.313369,-0.651804,1.938794,-0.112795,-0.868143,-0.451747,-1.052877,-0.468428,...,0.646261,0.839728,-1.185429,-0.475042,-1.238475,-1.001792,0.000000,1.003673e-17,1.111175e-17,-0.745439
5,-6.088086e-01,-1.009761,-0.539646,0.918192,-1.291464,1.917156,1.040948,-1.005763,0.508808,0.666766,...,-0.683404,0.998266,-0.029860,1.239982,0.577488,2.067499,0.121800,-1.742045e+00,-4.559336e-01,1.230371
6,5.098688e-01,-0.576543,0.651504,0.757735,0.725371,-0.735060,1.275085,2.288737,1.274570,-0.439827,...,0.549027,-0.555730,-0.100206,-0.790677,0.666224,1.044402,0.812469,6.272329e-01,1.040816e+00,1.089751
7,6.266844e-01,-1.098374,0.331435,-0.451288,0.669171,-0.780810,-0.175049,-1.269749,-0.985852,1.788886,...,0.415428,-0.765461,-0.591046,-0.966153,-0.177852,0.021305,-1.152798,-5.147361e-01,-1.218187e+00,-0.432856
8,-3.083335e-01,0.037323,0.485126,-0.842157,0.678112,-0.656806,-0.804620,-0.902773,-1.036958,0.013412,...,0.181432,1.013129,-0.119358,0.018827,-0.632361,-1.001792,0.000000,1.003673e-17,1.111175e-17,-0.745439
9,1.174312e-01,1.123253,-0.291707,2.128582,-0.975975,0.446346,0.835660,-0.992741,0.558239,0.711162,...,-0.235176,-0.374073,2.765216,0.471863,0.671497,0.021305,1.764019,-1.998790e+00,-1.780446e+00,0.965872


## Do your crazy machine learning thing here :) ...

## data visualization

In [ ]:
signaldata = ZData[np.nonzero(y == 1)]
nosignaldata = ZData[np.nonzero(y == -1)]

In [ ]:
for i in range(ZData.shape[1]):
    plt.figure
    plt.hist(signaldata[:,i],100,color='b')
    plt.hist(nosignaldata[:,i],100,color='r')
    plt.show()

In [ ]:
cov = np.corrcoef(np.transpose(ZData))
plt.imshow(cov)
plt.colorbar()

## modeling

## Simple model

In [39]:
initial_w = np.zeros(ZData.shape[1])
max_iters = 100
gamma = 0.1
losses, w = gradient_descent(y, ZData, initial_w, max_iters, gamma)
weights = np.asarray(w)[-1,:]

Gradient Descent(0/99): loss=0.5, w0=0.0010435125394016706, w1=-0.0333576512915564
Gradient Descent(10/99): loss=0.42169346688319354, w0=0.002620636760796259, w1=-0.16960356004291988
Gradient Descent(20/99): loss=0.4145852534707185, w0=-0.0049679972196410085, w1=-0.20565381864024662
Gradient Descent(30/99): loss=0.41118336432970454, w0=-0.012072773429225761, w1=-0.2212757296432853
Gradient Descent(40/99): loss=0.4089624058313634, w0=-0.016873353042579444, w1=-0.23027714859371184
Gradient Descent(50/99): loss=0.4073859704682621, w0=-0.01967759390795485, w1=-0.23631611243276937
Gradient Descent(60/99): loss=0.4062232631728083, w0=-0.02098025644259114, w1=-0.24072022239321725
Gradient Descent(70/99): loss=0.4053448952911179, w0=-0.021196916214167357, w1=-0.24409461125955684
Gradient Descent(80/99): loss=0.40466978083513633, w0=-0.020650446832853234, w1=-0.24676428510651874
Gradient Descent(90/99): loss=0.4041440212455612, w0=-0.019586350165745964, w1=-0.24892521659616765


In [40]:
np.transpose(init_w).shape
print(weights)

[-1.83393118e-02 -2.50541616e-01 -1.66602856e-01  4.34857664e-02
  2.24042804e-01 -4.45512529e-02  3.58026472e-02 -1.26378341e-01
  1.33395470e-01  1.82224322e-01 -1.04177994e-03 -1.42703983e-03
  1.83190190e-01 -8.14311218e-04  2.57990623e-03  8.01294773e-02
  1.15305870e-03 -5.57927516e-02  2.68600732e-02 -3.06258528e-02
 -2.05495590e-05  5.59959238e-04 -4.06415526e-02]


In [ ]:
print(ZData.shape)
seed = 1
degree = 7
k_fold = 4
lambdas = np.logspace(-4, 0, 30)
# split data in k fold
k_indices = build_k_indices(y, k_fold, seed)
# define lists to store the loss of training data and test data
rmse_tr = np.zeros(len(lambdas))
rmse_te = np.zeros(len(lambdas))

for i in range(0, len(lambdas)):
    rmse_tr[i], rmse_te[i] = cross_validation(y, ZData, k_indices, k_fold, lambdas[i], degree)

cross_validation_visualization(lambdas, rmse_tr, rmse_te)

## Generate predictions and save ouput in csv format for submission:

In [49]:
DATA_TEST_PATH = '/Users/benoithohl/Desktop/epfl/master_epfl/Ma3/Machine_learning/AIAIaie/data/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)
DataTest = remove_features_with_too_many_missing_values(tX_test,0.66)
DataTest = replace_missing_values_with_global_mean(DataTest)
ZDataTest = Z_score_of_each_feature(DataTest)


7 features removed (features number:  [ 4  5  6 12 26 27 28] )
new shape of data: (568238, 23)
range(0, 23)


In [50]:
OUTPUT_PATH = '/Users/benoithohl/Desktop/epfl/master_epfl/Ma3/Machine_learning/AIAIaie/data/predictions.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, ZDataTest)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)